# CNN Example code
## Please install required module first
### You need to install opencv, numpy, matplotlib

In [ ]:
import opencv as cv2
import numpy as np
import matplotlib as plt

# Open file \& Read image

In [ ]:
cv2.readimg()